In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from ELMO import ELMo
import torch.nn.functional as F
from sts_loader import STSDataset
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
print(device)

In [ ]:
# Load vocabularies
vocab = torch.load('Hin_vocab.pt')
character_vocab = torch.load('Hin_character_vocab.pt')

In [ ]:
# Get Sentences and Scores

path = 'sts-train-hi.tsv'
sts_dataset = STSDataset(path)
s1, s2, scores = sts_dataset.format(character_vocab)
scores = torch.tensor(scores, dtype=torch.float32)

In [ ]:
def create_dataloader(s1, s2, scores, batch_size):
    zipped = list(zip(s1, s2, scores))
    dataloader = DataLoader(zipped, batch_size=batch_size, shuffle=True)
    return dataloader

In [ ]:
# make the dataloader
s1_val = s1[:1000]
s2_val = s2[:1000]
scores_val = scores[:1000]
batch_size = 64
val_dataloader = create_dataloader(s1_val, s2_val, scores_val, batch_size)
train_dataloader = create_dataloader(s1[1000:], s2[1000:], scores[1000:], batch_size)

In [ ]:
# s1[1]

In [ ]:
model = ELMo(cnn_config = {'character_embedding_size': 16, 
                           'num_filters': 32, 
                           'kernel_size': 5, 
                           'max_word_length': 10, 
                           'char_vocab_size': len(character_vocab)}, 
             elmo_config = {'num_layers': 3,
                            'word_embedding_dim': 300,
                            'vocab_size': len(vocab)}, 
             char_vocab_size = len(character_vocab)).to(device)


In [ ]:
model.load_state_dict(torch.load('Hin_model.pt'))

In [1]:
word_embedding_dim = 300

# make a model for sentence similairty
class SimilarityModel(nn.Module):
    def __init__(self, elmo):
        super(SimilarityModel, self).__init__()
        self.elmo = elmo
        self.lstm = nn.LSTM(word_embedding_dim, word_embedding_dim/2, batch_first=True, bidirectional=True)
        
    def forward(self, sentence1, sentence2):
        _, sentence1 = self.elmo(sentence1)
        _, sentence2 = self.elmo(sentence2)
        return (F.cosine_similarity(sentence1.mean(dim=1), sentence2.mean(dim=1)) + 1)*(5/2)

NameError: name 'nn' is not defined

In [ ]:
# similarity_model = SimilarityModel(model)
similarity_model = SimilarityModel(model, 128).to(device)

In [ ]:
num_epochs = 10
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(similarity_model.parameters(), lr=0.001)


In [ ]:
# train the model
for epoch in range(num_epochs):
    similarity_model.train()
    total_loss = 0
    for s1, s2, scores in tqdm(train_dataloader):
        s1 = s1
        s2 = s2
        scores = scores.to(device)
        optimizer.zero_grad()
        outputs = similarity_model(s1, s2)
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch} Loss: {total_loss}")

    similarity_model.eval()
    with torch.no_grad():
        total_loss = 0
        for s1, s2, scores in val_dataloader:
            s1 = s1
            s2 = s2
            scores = scores.to(device)
            outputs = similarity_model(s1, s2)
            loss = criterion(outputs, scores)
            total_loss += loss.item()
        print(f"Validation Loss: {total_loss}")

In [ ]:
# iterate over the dataloader